In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tqdm")
warnings.filterwarnings("ignore", category=UserWarning, module="dask.dataframe._pyarrow_compat")

In [1]:
from Albert import Einstein 
from Stephen import Hawking 
from transformers import GenerationConfig, AutoTokenizer

import os
from dotenv import load_dotenv
load_dotenv()

/home/sabankara/.conda/envs/meta/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 12.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


True

In [4]:
import torch
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
from langchain.prompts.prompt import PromptTemplate
import warnings
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms.base import LLM
warnings.filterwarnings('ignore')


Einstein_Model= "/home/sabankara/coding/physics/einstein_file"
Einstein_Model= "/home/sabankara/coding/physics/hawking_file"

instance_ = Einstein()
_ , tokenizer_ =instance_.huggingface_model_load()

instance = Hawking()
_ , tokenizer=instance.huggingface_model_load()

einstein_pipe =transformers.pipeline(
    "text2text-generation",
    model=Einstein_Model,
    tokenizer=tokenizer_,
    device_map= {"":0},
    torch_dtype=torch.bfloat16,
    min_new_tokens= 80,
    max_new_tokens=200,
    num_beams=1,
    do_sample=True,
    top_p=0.9,
    top_k=0,
    temperature=0.5,
    repetition_penalty=2.5
    )

hawking_pipe =transformers.pipeline(
    "text2text-generation",
    model=Einstein_Model,
    tokenizer=tokenizer_,
    device_map= {"":0},
    torch_dtype=torch.bfloat16,
    min_new_tokens= 40,
    num_beams=1,
    do_sample=True,
    top_p=0.5,
    top_k=0,
    temperature=0.5,
    repetition_penalty=2.5
    )

eintein_llm=HuggingFacePipeline(pipeline=einstein_pipe)
hawking_llm=HuggingFacePipeline(pipeline=hawking_pipe)

In [83]:
template_ei = """\n\nAnswer the following question:\n\n{input}\n\nAnswer:{history}"""
PROMPT_ = PromptTemplate(input_variables=["history","input"], template=template_ei)

# memory = ConversationBufferWindowMemory(k=3)
memory_ = ConversationBufferMemory()
conversation1 = ConversationChain(
    prompt=PROMPT_,
    llm=eintein_llm, 
    memory = memory_,
    verbose=False
)

template_hw = """\n\nAsk question based on answer:\n\nAnswer\n\n{input}\n\nQuestion:{history}"""
PROMPT = PromptTemplate(input_variables=["history","input"], template=template_ei)

# memory = ConversationBufferWindowMemory(k=3)
memory = ConversationBufferMemory()
conversation2 = ConversationChain(
    prompt=PROMPT,
    llm=hawking_llm, 
    memory = memory,
    verbose=False
)


question = "How do black holes affect the cosmic neutrino background and what are the observable consequences of these effects?"
for _ in range(3):
    einstein_response = conversation1.predict(input=question)
    
    hawking_response = conversation2.predict(input=einstein_response)
    
    input_text = hawking_response

    print("Einstein:", einstein_response)
    print("Hawking:", hawking_response)


Einstein: The black holes are a source of radiation that can be detected by the neutrino. Black holes also affect the plasma membrane and their environment, which is very sensitive to sunlight and heat. These effects occur in high-energy clouds and very low temperature areas. These particles have the ability to penetrate deep into our atmosphere and cause ultraviolet light to shine through the surface of the planet. These effect is caused by an increase in atmospheric pressure and less pollution from which we live.
Hawking: black holes are a source of radiation that can be detected by the neutrino. Black
Einstein: Human: Black holes also affect the plasma membrane and their environment, which is very sensitive to sunlight and heat. These effects occur in high-energy clouds and very low temperature areas. These particles have the ability to penetrate deep into our atmosphere and cause ultraviolet light to shine through the surface of the planet. These effect is caused by an increase in 

# USE WİTHOUT LANGCHAIN

In [13]:
instance = Hawking()
_ , tokenizer =instance.huggingface_model_load()
model_hawking = instance.load_model(os.getenv("hawking_path"))

def hawking_answer(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda(0)
    model_outputs = model_hawking.generate(input_ids=input_ids, generation_config=GenerationConfig(min_new_tokens= 40, num_beams=1, do_sample=True, top_p=0.9, top_k=0, temperature=0.8,repetition_penalty=2.5))
    model_text_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)
    return model_text_output

instance_ = Einstein()
_ , tokenizer =instance_.huggingface_model_load()
model_einstein = instance_.load_model(os.getenv("einstein_path"))

def einstein_answer(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda(0)
    model_outputs = model_einstein.generate(input_ids=input_ids, generation_config=GenerationConfig(min_new_tokens= 80, max_new_tokens=250, num_beams=1, do_sample=True, top_p=0.6, top_k=0, temperature=0.4,repetition_penalty=2.5))

    model_text_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)
    return model_text_output

In [16]:
import sys
import time
prompt="Ask question based on answer:\n\nAnswer\n\nA black hole is a region in space where the gravitational force is so strong that even light cannot escape. This region may form as a result of the collapse of a star. Square holes absorb the matter around them, sucking everything, including light, into space. They are called black because even light cannot escape. Because light cannot withstand the gravitational force and is pulled in. Black holes are mysterious and complex phenomena because they are difficult to observe..\n\nQuestion"

prompt_list = ["Ask a comprehensive question about 'Dark Matter'\n","Ask a comprehensive question about 'Black holes'\n", "Ask a comprehensive question about 'String theory'\n"]

prompt2 = "\n\nAnswer the following question:\n\n{x}\n\nAnswer"

input_text = prompt

def print_character_by_character(text, delay=0.005, color_code="\033[36m"):
    for char in text:
        sys.stdout.write(f"{color_code}{char}\033[0m")
        sys.stdout.flush()
        time.sleep(delay)

def add_newline_after_punctuation(text):
    n=17
    words = text.split()
    updated_text = ""

    for i, word in enumerate(words):
        updated_text += word + " "
        if (i + 1) % n == 0:
            updated_text += "\n"

    return updated_text.strip()

for _ in range(3): 
    stephen_response = hawking_answer(prompt_list[_])

    albert_response = einstein_answer(prompt2.format(x=stephen_response))

    input_text = albert_response

    albert_response = add_newline_after_punctuation(albert_response)

    print("\033[32mHawking:\033[0m")

    print_character_by_character(stephen_response)
    sys.stdout.write("\n")

    print("\033[32mEintein:\033[0m")
    print_character_by_character(albert_response)
    sys.stdout.write("\n")



Hawking:
"What is the secret of dark matter?
Eintein:
Dark matter is a chemical that has been found in the atmosphere, and it can be used 
to describe dark objects. The secret of dark matter is known as "the light" (or simply "light"), 
which means that there are many types of particles that have formed or evolved from other materials. 
This process involves creating an invisible layer of black-like material with no visible wavelengths. In this case, 
we know that dark matter is one of the most powerful molecules in our solar system, but 
it's not always clear what they do. However, dark matter is also believed to contain ultraviolet radiation, 
such as UVA radiation. It could cause some damage to the surface, especially when the sun goes 
off. These layers can lead to darkness, including those around us who believe dark matter may exist. 
To understand how dark matter affects our environment, you should consider several factors: 1) A lack of 
energy, or the ability to produce new thin